# Étape 1 : Importation des bibliothèques et préparation des données

In [4]:
# Importer les bibliothèques nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re  # Pour le nettoyage de texte
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.metrics import classification_report

In [6]:
# Charger le dataset
dataset_path = "/content/sentiment100k.csv"
data = pd.read_csv(dataset_path,  names=['polarity', 'id', 'date', 'query', 'user', 'text'], encoding='latin-1') # encodage latin-1 est souvent utilisé pour des fichiers contenant des caractères spéciaux.

# Afficher les premières lignes pour vérifier le chargement
print("Aperçu des données :")
data.head()

Aperçu des données :


,polarity,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [7]:
# Garder uniquement les colonnes 'polarity' et 'text'
data = data[['polarity', 'text']]

# Convertir les labels : 0 (négatif) et 1 (positif)
data['polarity'] = data['polarity'].apply(lambda x: 0 if x == 0 else 1)

# supprimer les doublons et les valeurs manquantes
data.drop_duplicates(inplace=True)
data.dropna(inplace=True)

# Afficher un résumé des données après nettoyage
print("\nRésumé des données nettoyées :")
print(data.info())


Résumé des données nettoyées :
<class 'pandas.core.frame.DataFrame'>
Index: 198671 entries, 0 to 199999
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   polarity  198671 non-null  int64 
 1   text      198671 non-null  object
dtypes: int64(1), object(1)
memory usage: 4.5+ MB
None


<ipython-input-7-0ec1ce9385d1>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['polarity'] = data['polarity'].apply(lambda x: 0 if x == 0 else 1)


In [8]:
# Aperçu des premières lignes après nettoyage
print("\nPremières lignes des données nettoyées :")
data.head()


Premières lignes des données nettoyées :


,polarity,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


# Prétraitement des Textes

In [9]:
# Fonction de nettoyage des tweets
def clean_text(text):
    # Convertir en minuscules
    text = text.lower()
    # Supprimer les URL
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    # Supprimer les caractères non alphabétiques
    text = re.sub(r"[^a-zA-Z\s]", '', text)
    # Supprimer les espaces en trop
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [10]:
# Appliquer la fonction de nettoyage à la colonne 'text'
data['text'] = data['text'].apply(clean_text)
data.head()

,polarity,text
0,0,switchfoot a thats a bummer you shoulda got da...
1,0,is upset that he cant update his facebook by t...
2,0,kenichan i dived many times for the ball manag...
3,0,my whole body feels itchy and like its on fire
4,0,nationwideclass no its not behaving at all im ...


In [11]:
print("\nExemple de données nettoyées :")
data["text"][0]


Exemple de données nettoyées :


'switchfoot a thats a bummer you shoulda got david carr of third day to do it d'

# Tokenisation et Padding

In [12]:
# Paramètres pour la tokenisation et le padding
max_vocab_size = 10000  # Nombre maximum de mots dans le vocabulaire
max_sequence_length = 100  # Longueur maximale des séquences

# Initialisation du tokenizer
tokenizer = Tokenizer(num_words=max_vocab_size, oov_token="<OOV>")  # "<OOV>" pour les mots hors vocabulaire

# Adapter le tokenizer aux textes
tokenizer.fit_on_texts(data['text'])

# Convertir les textes en séquences de tokens
sequences = tokenizer.texts_to_sequences(data['text'])

# Appliquer le padding aux séquences
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post', truncating='post')

In [13]:
print("\nExemple d'une séquence tokenisée et paddée :")
padded_sequences[0]


Exemple d'une séquence tokenisée et paddée :


array([   1,    5,  103,    5, 1333,    8, 3908,   51,  768, 7637,   13,
       2003,   32,    3,   40,   10,  414,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0], dtype=int32)

# Séparation des Données + Construction du Modèle

In [18]:
# 1. Séparation des données
X_train, X_test, y_train, y_test = train_test_split(
    padded_sequences, data['polarity'], test_size=0.2, random_state=42
)

In [19]:
# 2. Construction du modèle séquentiel
model = Sequential([
    # Couche d'embedding
    Embedding(input_dim=10000, output_dim=128, input_length=100),
    # Couche LSTM
    LSTM(128, dropout=0.2, recurrent_dropout=0.2),
    # Couche Dense (binaire)
    Dense(1, activation='sigmoid')  # Activation sigmoid pour une sortie binaire
])

In [20]:
# 3. Compilation du modèle
model.compile(
    loss='binary_crossentropy',  # Fonction de perte binaire
    optimizer='adam',            # Optimiseur Adam
    metrics=['accuracy']         # Suivi de la précision
)

In [21]:
# Résumé du modèle
print("\nRésumé du modèle :")
model.summary()


Résumé du modèle :


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

# Entraînement et Évaluation du Modèle

In [22]:
# 1. Entraînement du modèle
history = model.fit(
    X_train, y_train,
    epochs=5,
    batch_size=32,
    validation_data=(X_test, y_test),
    verbose=2
)

Epoch 1/5
4967/4967 - 853s - 172ms/step - accuracy: 0.5011 - loss: 0.6934 - val_accuracy: 0.4993 - val_loss: 0.6932
Epoch 2/5
4967/4967 - 845s - 170ms/step - accuracy: 0.4989 - loss: 0.6933 - val_accuracy: 0.5007 - val_loss: 0.6932
Epoch 3/5
4967/4967 - 839s - 169ms/step - accuracy: 0.5001 - loss: 0.6932 - val_accuracy: 0.5007 - val_loss: 0.6931
Epoch 4/5
4967/4967 - 833s - 168ms/step - accuracy: 0.4999 - loss: 0.6932 - val_accuracy: 0.5007 - val_loss: 0.6932
Epoch 5/5
4967/4967 - 862s - 173ms/step - accuracy: 0.4994 - loss: 0.6932 - val_accuracy: 0.4993 - val_loss: 0.6932


In [23]:
# 2. Évaluation du modèle sur les données de test
loss, accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"\nPerte sur les données de test : {loss:.4f}")
print(f"Précision sur les données de test : {accuracy:.4f}")

1242/1242 - 51s - 41ms/step - accuracy: 0.4993 - loss: 0.6932

Perte sur les données de test : 0.6932
Précision sur les données de test : 0.4993


In [25]:
# 3. Prédictions et rapport de classification
y_pred = (model.predict(X_test) > 0.5).astype("int32")  # Convertir les probabilités en classes
print("\nRapport de classification :")
print(classification_report(y_test, y_pred, target_names=['Négatif', 'Positif']))

1242/1242 ━━━━━━━━━━━━━━━━━━━━ 53s 42ms/step

Rapport de classification :
              precision    recall  f1-score   support

     Négatif       0.00      0.00      0.00     19895
     Positif       0.50      1.00      0.67     19840

    accuracy                           0.50     39735
   macro avg       0.25      0.50      0.33     39735
weighted avg       0.25      0.50      0.33     39735



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Prédiction de Sentiment pour de Nouveaux Tweets

In [26]:
def preprocess_text(text, tokenizer, max_sequence_length):
    # Convertir le texte en minuscules
    text = text.lower()
    # Nettoyer les URL, les caractères non alphabétiques et les espaces en trop
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)  # Supprimer les URL
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Supprimer les caractères non alphabétiques
    text = text.strip()  # Supprimer les espaces inutiles

    # Tokenisation et padding
    sequence = tokenizer.texts_to_sequences([text])  # Convertir en séquence de tokens
    padded_sequence = pad_sequences(sequence, maxlen=max_sequence_length, padding='post', truncating='post')

    return padded_sequence

In [27]:
# Exemple de tweet
tweet = "I love this product! It's amazing! #happy"

# Prétraitement du texte
processed_tweet = preprocess_text(tweet, tokenizer, max_sequence_length)

# Prédiction du sentiment
prediction = model.predict(processed_tweet)

# Interprétation du résultat
sentiment = "Positif" if prediction > 0.5 else "Négatif"
print(f"Le sentiment du tweet est : {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
Le sentiment du tweet est : Positif
